In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import keplergl
#from keplergl import KeplerGL
import matplotlib as plt
import plotly.express as px
import h3
import shapely
from geopandas import GeoDataFrame,GeoSeries
import folium
import matplotlib
%matplotlib inline

In [26]:
road_network = gpd.read_file('Ontario_Road_Network_(ORN)_Segment_With_Address (1).geojson')

In [33]:
#convert column names to lowercase
road_network.columns = [s.strip().lower() for s in road_network.columns]

In [34]:
road_network.shape

(607277, 38)

In [35]:
road_network.columns

Index(['ogf_id', 'official_street_name', 'abbreviated_street_name',
       'alternate_street_name', 'directional_prefix', 'street_type_prefix',
       'street_name_body', 'street_type_suffix', 'directional_suffix',
       'road_class', 'road_element_type', 'l_first_house_number',
       'l_last_house_number', 'l_house_number_structure',
       'l_original_municipality', 'l_standard_municipality',
       'r_first_house_number', 'r_last_house_number',
       'r_house_number_structure', 'r_original_municipality',
       'r_standard_municipality', 'direction_of_traffic_flow', 'route_name_1',
       'route_name_2', 'route_name_3', 'route_name_4', 'route_number_1',
       'route_number_2', 'route_number_3', 'shield_type', 'length',
       'geometry_update_datetime', 'effective_datetime', 'system_datetime',
       'objectid', 'shapelen', 'shape', 'geometry'],
      dtype='object')

In [36]:
#filter only toronto roads
road_network = road_network[road_network['r_standard_municipality'] == 'City of Toronto']

In [37]:
#check dataframe shape of remaining rows
road_network.shape

(50066, 38)

In [38]:
#dropping unneccessary columns

#specifying columns to keep
cols_to_keep = ['official_street_name','road_class','geometry']

#dropping columns
road_network = road_network[cols_to_keep]

In [39]:
road_network.head()

,official_street_name,road_class,geometry
14,JOHN TABOR TRAIL,Collector,"LINESTRING (-79.20635 43.80700, -79.20645 43.8..."
17,HILLCREST AVENUE,Local / Street,"LINESTRING (-79.40732 43.76861, -79.40474 43.7..."
24,KINGSTON ROAD,Arterial,"LINESTRING (-79.25200 43.70537, -79.25138 43.7..."
68,IAN MACDONALD BOULEVARD,Local / Strata,"LINESTRING (-79.49831 43.77091, -79.49836 43.7..."
161,SARGENT LANE,Local / Street,"LINESTRING (-79.29759 43.68485, -79.29664 43.6..."


In [40]:
#extracting latitude and longitude values from the linestring centroids
road_network['latitude'] = road_network.apply(lambda x: x.geometry.centroid.coords[0][1], axis=1)
road_network['longitude'] = road_network.apply(lambda x: x.geometry.centroid.coords[0][0], axis=1)

In [41]:
#combining latitude and longitudes to form coordinates column
road_network['coordinates'] = road_network.apply(lambda x: (x.latitude, x.longitude), axis=1)

#create hex cells for bike lanes

#create coordinate column
#pedestrian_crossover_exploded['coordinates'] = pedestrian_crossover_exploded['latitude'].astype('str') + ',' + pedestrian_crossover_exploded['longitude'].astype('str')

#then we define the function to get h3 hex cells from coordinates data
def get_hex_cell(coordinates):
    #a = float(coordinates.split(',')[0])
    #b = float(coordinates.split(',')[1])
    a = coordinates[0]
    b = coordinates[1]
    return h3.latlng_to_cell(a,b,res=10)

#finally, create a new column for hex cells by applying the function on coordinates column
road_network['hex_cell'] = road_network['coordinates'].apply(get_hex_cell)

In [42]:
road_network.head()

,official_street_name,road_class,geometry,latitude,longitude,coordinates,hex_cell
14,JOHN TABOR TRAIL,Collector,"LINESTRING (-79.20635 43.80700, -79.20645 43.8...",43.807649,-79.206579,"(43.807649211684385, -79.2065785292571)",8a2b9bd60aeffff
17,HILLCREST AVENUE,Local / Street,"LINESTRING (-79.40732 43.76861, -79.40474 43.7...",43.768902,-79.406031,"(43.76890200429669, -79.40603095144428)",8a2b9bc2cc07fff
24,KINGSTON ROAD,Arterial,"LINESTRING (-79.25200 43.70537, -79.25138 43.7...",43.705879,-79.251689,"(43.705879350406875, -79.25168929659739)",8a2b9bd48287fff
68,IAN MACDONALD BOULEVARD,Local / Strata,"LINESTRING (-79.49831 43.77091, -79.49836 43.7...",43.771719,-79.498648,"(43.771719009135126, -79.49864758217092)",8a2b9bc044cffff
161,SARGENT LANE,Local / Street,"LINESTRING (-79.29759 43.68485, -79.29664 43.6...",43.684958,-79.297116,"(43.68495809463395, -79.29711570926605)",8a2b9bc6476ffff


In [43]:
road_network.to_csv('road_network_processed.csv')

# Intersections

In [2]:
intersections = gpd.read_file('Centreline Intersection - 4326.geojson')

In [3]:
intersections.head()

,_id,INTERSECTION_ID,DATE_EFFECTIVE,DATE_EXPIRY,ELEVATION_ID,INTERSECTION_DESC,CLASSIFICATION,CLASSIFICATION_DESC,NUMBER_OF_ELEVATIONS,ELEVATION_FEATURE_CODE,...,ELEVATION_LEVEL,ELEVATION,ELEVATION_UNIT,HEIGHT_RESTRICTION,HEIGHT_RESTRICTION_UNIT,STATE,TRANS_ID_CREATE,TRANS_ID_EXPIRE,OBJECTID,geometry
0,1,13470264,2008-12-12 04:22:46,3000-01-01T05:00:00,13,Robindale Ave / Rimilton Ave,MNRSL,Minor-Single Level,1,501300,...,0,0.0,None,0.0,None,8,200000,-1,1,MULTIPOINT (-79.53107 43.60724)
1,2,13470193,2008-12-12 04:22:46,3000-01-01T05:00:00,4718,Bellman Ave / Valermo Dr,MNRSL,Minor-Single Level,1,501300,...,0,0.0,None,0.0,None,8,200000,-1,4,MULTIPOINT (-79.53137 43.60960)
2,3,13470188,2008-12-12 04:22:46,3000-01-01T05:00:00,32728,Rimilton Ave / Valermo Dr,SEUSL,Pseudo Intersection-Single Level,1,509200,...,0,0.0,None,0.0,None,8,200000,-1,5,MULTIPOINT (-79.53012 43.60983)
3,4,13470203,2008-12-12 04:22:46,3000-01-01T05:00:00,21669,Valermo Dr / Goa Crt,MNRSL,Minor-Single Level,1,501300,...,0,0.0,None,0.0,None,8,200000,-1,7,MULTIPOINT (-79.53317 43.60919)
4,5,13470228,2008-12-12 04:22:46,3000-01-01T05:00:00,36820,Valermo Dr / Thirtieth St,MNRSL,Minor-Single Level,1,501300,...,0,0.0,None,0.0,None,8,200000,-1,9,MULTIPOINT (-79.53559 43.60864)


In [4]:
intersections.columns = [s.strip().lower() for s in intersections.columns]

In [5]:
intersections.columns

Index(['_id', 'intersection_id', 'date_effective', 'date_expiry',
       'elevation_id', 'intersection_desc', 'classification',
       'classification_desc', 'number_of_elevations', 'elevation_feature_code',
       'elevation_feature_code_desc', 'elevation_level', 'elevation',
       'elevation_unit', 'height_restriction', 'height_restriction_unit',
       'state', 'trans_id_create', 'trans_id_expire', 'objectid', 'geometry'],
      dtype='object')

In [6]:
#dropping unneccessary columns
cols_to_keep = ['intersection_id','intersection_desc','geometry']
intersections = intersections[cols_to_keep]

In [7]:
#set intersection_id as id index
intersections.set_index('intersection_id',inplace=True)      

In [8]:
intersections.head()

,intersection_desc,geometry
intersection_id,,
13470264,Robindale Ave / Rimilton Ave,MULTIPOINT (-79.53107 43.60724)
13470193,Bellman Ave / Valermo Dr,MULTIPOINT (-79.53137 43.60960)
13470188,Rimilton Ave / Valermo Dr,MULTIPOINT (-79.53012 43.60983)
13470203,Valermo Dr / Goa Crt,MULTIPOINT (-79.53317 43.60919)
13470228,Valermo Dr / Thirtieth St,MULTIPOINT (-79.53559 43.60864)
